In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random


In [6]:
D= 0.039
d= 0.0078
H= 0.42
rho= 0.7356
mu= 26.09E-6
nu=mu/rho
u_in=1

In [7]:
NN = tf.keras.models.Sequential([
    tf.keras.layers.Input((3,)),
    tf.keras.layers.Dense(units = 64, activation = 'tanh'),
    tf.keras.layers.Dense(units = 64, activation = 'tanh'),
    tf.keras.layers.Dense(units = 64, activation = 'tanh'),
    tf.keras.layers.Dense(units = 64, activation = 'tanh'),
    tf.keras.layers.Dense(units = 64, activation = 'tanh'),
    tf.keras.layers.Dense(units = 4)
])

NN.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,156 (67.02 KB)

 Trainable params: 17,156 (67.02 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
optm = tf.keras.optimizers.Adam(learning_rate = 0.001)


In [9]:
def pde_loss(x, y, z, net, nu, rho):
    x = x.reshape(-1,1)
    x = tf.constant(x, dtype = tf.float32)  
    y = y.reshape(-1,1)
    y = tf.constant(y, dtype = tf.float32) 
    z = z.reshape(-1,1)
    z = tf.constant(z, dtype = tf.float32) 

    # Compute derivatives
    with tf.GradientTape(persistent=True) as g:
        g.watch(x)
        g.watch(y)
        g.watch(z)
        with tf.GradientTape() as gg:
            gg.watch(x)
            gg.watch(y)
            gg.watch(z)
            u = net(tf.concat([x, y, z], axis=1))[:,0]
        u_x, u_y, u_z = g.gradient(u, [x, y, z])
    u_xx = g.gradient(u_x, x)
    u_yy = g.gradient(u_y, y)
    u_zz = g.gradient(u_z, z)
    with tf.GradientTape(persistent=True) as g:
        g.watch(x)
        g.watch(y)
        g.watch(z)
        with tf.GradientTape() as gg:
            gg.watch(x)
            gg.watch(y)
            gg.watch(z)
            v = net(tf.concat([x, y, z], axis=1))[:,1]
        v_x, v_y, v_z = g.gradient(v, [x, y, z])
    v_xx = g.gradient(v_x, x)
    v_yy = g.gradient(v_y, y)
    v_zz = g.gradient(v_z, z)
    with tf.GradientTape(persistent=True) as g:
        g.watch(x)
        g.watch(y)
        g.watch(z)
        with tf.GradientTape() as gg:
            gg.watch(x)
            gg.watch(y)
            gg.watch(z)
            w = net(tf.concat([x, y, z], axis=1))[:,2]
        w_x, w_y, w_z = g.gradient(w, [x, y, z])
    w_xx = g.gradient(w_x, x)
    w_yy = g.gradient(w_y, y)
    w_zz = g.gradient(w_z, z)
    with tf.GradientTape() as g:
        g.watch(x)
        g.watch(y)
        g.watch(z)            
        p = net(tf.concat([x, y, z], axis=1))[:,3]
        p_x, p_y, p_z = g.gradient(p, [x, y, z])
   

    # Compute PDE terms
    pde_u = u * u_x + v * u_y + w * u_z + 1/rho * p_x - nu * (u_xx + u_yy + u_zz)
    pde_v = u * v_x + v * v_y + w * v_z + 1/rho * p_y - nu * (v_xx + v_yy + v_zz)
    pde_w = u * w_x + v * w_y + w * w_z + 1/rho * p_z - nu * (w_xx + w_yy + w_zz)
    pde_0 = u_x+ v_y+ w_z
    

    # Compute square loss
    square_loss = tf.square(pde_u) + tf.square(pde_v) + tf.square(pde_w) + tf.square(pde_0)
    total_loss = tf.reduce_mean(square_loss)

    return total_loss
def BC_loss(x, y, z, net):
    x = x.reshape(-1,1)
    x = tf.constant(x, dtype = tf.float32)  
    y = y.reshape(-1,1)
    y = tf.constant(y, dtype = tf.float32) 
    z = z.reshape(-1,1)
    z = tf.constant(z, dtype = tf.float32) 
    # Compute boundary conditions
    # At the cylinder body
    bc_u =  net(tf.concat([x, y, z], axis=1))[:,0]
    bc_v =  net(tf.concat([x, y, z], axis=1))[:,1]
    bc_w =  net(tf.concat([x, y, z], axis=1))[:,2]
    square_loss= tf.square(bc_u) + tf.square(bc_v)+ tf.square(bc_w)
    total_loss = tf.reduce_mean(square_loss)

    return total_loss


def IN_loss(x, y, z, net, u_in):

    x = x.reshape(-1,1)
    x = tf.constant(x, dtype = tf.float32)  
    y = y.reshape(-1,1)
    y = tf.constant(y, dtype = tf.float32) 
    z = z.reshape(-1,1)
    z = tf.constant(z, dtype = tf.float32) 
    # At the inlet
    bc_inlet_u = net(tf.concat([x, y, z], axis=1))[:,0] 
    bc_inlet_v = net(tf.concat([x, y, z], axis=1))[:,1]
    bc_inlet_w = net(tf.concat([x, y, z], axis=1))[:,2]-u_in
    square_loss= tf.square(bc_inlet_u) + tf.square(bc_inlet_v)+ tf.square(bc_inlet_w)
    total_loss = tf.reduce_mean(square_loss)
    return total_loss
def OUT_loss(x, y, z, net):
    x = x.reshape(-1,1)
    x = tf.constant(x, dtype = tf.float32)  
    y = y.reshape(-1,1)
    y = tf.constant(y, dtype = tf.float32) 
    z = z.reshape(-1,1)
    z = tf.constant(z, dtype = tf.float32) 
    # At the outlet
    bc_outlet_p = net(tf.concat([x, y, z], axis=1))[:,3]
    square_loss= tf.square(bc_outlet_p)
    total_loss = tf.reduce_mean(square_loss)
    return total_loss

In [10]:
def generate_points_in_cylinder(R, H, N_pde):
    points = []
    while len(points) < N_pde:
        x = random.uniform(-R, R)
        y = random.uniform(-R, R)
        z = random.uniform(0, H)
        if x**2 + y**2 <= R**2:
            points.append((x, y, z))
    return np.array(points)

In [11]:
def generate_points_inlet(R, N_in):
    points = []
    while len(points) < N_in:
        x = random.uniform(-R, R)
        y = random.uniform(-R, R)
        z = random.uniform(0, 0+1e-4)
        if x**2 + y**2 <= R**2:
            points.append((x, y, z))
    return np.array(points)
def generate_points_outlet(R, H, N_out):
    points = []
    while len(points) < N_out:
        x = random.uniform(-R, R)
        y = random.uniform(-R, R)
        z = random.uniform(H-1e-4, H)
        if x**2 + y**2 <= R**2:
            points.append((x, y, z))
    return np.array(points)
def generate_points_BC(R, H, N_BC):
    points = []
    while len(points) < N_BC:
        x = random.uniform(-R, R)
        y = random.uniform(-R, R)
        z = random.uniform(0, H)
        if  R**2 -x**2 -y**2 <R**2/3 and R**2 -x**2 -y**2>=0:
            points.append((x, y, z))
    return np.array(points)

In [12]:
N_pde = 800
train_pde = generate_points_in_cylinder(D/2, H, N_pde)
train_x_pde=train_pde[:,0]
train_y_pde=train_pde[:,1]
train_z_pde=train_pde[:,2]
N_in = 50
train_in = generate_points_inlet(D/2, N_in)
train_x_in=train_in[:,0]
train_y_in=train_in[:,1]
train_z_in=train_in[:,2]
N_out = 50
train_out = generate_points_outlet(D/2, H, N_out)
train_x_out=train_out[:,0]
train_y_out=train_out[:,1]
train_z_out=train_out[:,2]
N_BC = 50
train_BC = generate_points_BC(D/2, H, N_BC)
train_x_BC=train_BC[:,0]
train_y_BC=train_BC[:,1]
train_z_BC=train_BC[:,2]

In [ ]:
fig=plt.figure()
ax= fig.add_subplot(111, projectio='3d')
ax.scatter(train_x_pde,train_y_pde,train_z_pde, c='b',marker='o')
plt.show()

AttributeError: Axes.set() got an unexpected keyword argument 'projectio'

In [ ]:
train_loss_record = []

for itr in range(3000):
    with tf.GradientTape() as tape:
        
        train_loss_BC= BC_loss (train_x_BC, train_y_BC, train_z_BC, NN)
        train_loss_IN= IN_loss (train_x_in, train_y_in, train_z_in, NN, u_in)
        
        train_loss_OUT= OUT_loss (train_x_out, train_y_out, train_z_out, NN)
        train_loss_pde = pde_loss(train_x_pde, train_y_pde, train_z_pde, NN, nu, rho)
        total_loss = train_loss_pde + train_loss_BC + train_loss_IN + train_loss_OUT
        
        train_loss_record.append(total_loss)
                
        grad_w = tape.gradient(total_loss, NN.trainable_variables)
        optm.apply_gradients(zip(grad_w, NN.trainable_variables))
    
    if itr % 1000 == 0:
        print(total_loss.numpy())
        
#plt.figure(figsize = (10,8))
#plt.plot(train_loss_record)
#plt.show()

2.2101166


In [ ]:
samples = np.random.rand(500000, 2)
samples[:, 0] = 0  # Set x-coordinate to 0
samples[:, 1] = (samples[:, 1] - 0.5) * D  # Scale y-coordinate to the desired range [-D/2, D/2]
samples = np.hstack([samples, np.random.rand(500000, 1) * H])
result = NN.predict(samples)

# Assuming you have defined `color_legend` elsewhere in your code
color_legend = [[0, 1.5], [-0.3, 0.3], [0, 35], [0, 35]]

# Plotting code (assuming `D` and `H` are defined)
for idx in range(4):
    plt.figure(figsize=(20, 4))
    plt.scatter(samples[:, 1],
                samples[:, 2],  # Assuming z coordinate is the third dimension
                c=result[:, idx],
                cmap='jet',
                s=2)
    plt.colorbar()
    #plt.clim(color_legend[idx])
    plt.xlim((-D/2, D/2))  # Adjusting x limits
    plt.ylim((0, H))       # Adjusting y limits
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_samples(N, D, H):
    samples = np.random.rand(N, 3)
    while True:
        samples[:, 0] = (np.random.rand(N) - 0.5) * D
        samples[:, 1] = (np.random.rand(N) - 0.5) * D
        samples[:, 2] = H/2
        mask = samples[:, 0]**2 + samples[:, 1]**2 <= (D/2)**2
        if np.sum(mask) >= N:
            break
    samples = samples[mask]
    return samples

N = 500000


samples = generate_samples(N, D, H)
result = NN.predict(samples)

# Assuming you have defined `color_legend` elsewhere in your code
color_legend = [[0, 1.5], [-0.3, 0.3], [0, 35], [0, 35]]

# Plotting code (assuming `D` and `H` are defined)
for idx in range(4):
    plt.figure(figsize=(20, 20))
    plt.scatter(samples[:, 0],
                samples[:, 1],  # Assuming z coordinate is the third dimension
                c=result[:, idx],
                cmap='jet',
                s=2)
    
    # Adding circle with radius D/2
    circle = plt.Circle((0, 0), D/2, color='black', fill=False)
    plt.gca().add_patch(circle)
    
    plt.colorbar()
    plt.xlim((-D/2, D/2))  # Adjusting x limits
    plt.ylim((-D/2, D/2))  # Adjusting y limits
    plt.tight_layout()
    plt.show()